In [1]:
import json
import pymongo
import tweepy

with open('consumer_key.txt', 'r') as f:
    consumer_key =  f.read()
f.closed

with open('consumer_secret.txt', 'r') as f:
    consumer_secret = f.read()
f.closed

with open('access_key.txt', 'r') as f:
    access_key = f.read()
f.closed

with open('access_secret.txt', 'r') as f:
     access_secret = f.read()
f.closed

# Authentication
auth = tweepy.OAuthHandler(consumer_key[:-1], consumer_secret[:-1])
auth.set_access_token(access_key[:-1], access_secret[:-1])
api = tweepy.API(auth)

In [2]:
# Test access

USER_NAME = "nurietta_17"
user = api.get_user(id=USER_NAME)
print (user.description)

VALAR MORGHULIS


In [49]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']
collection = db['tweets']

Connected successfully!!!


In [43]:
# Find 100 actual tweets located in Spain to store its user in mlab

collection = db['tweet_users']
    
search_results =api.search(q='*', geocode='39.3262345,-4.8380649,750km', count=100, lang='es')

json_users = []
print(len(search_results))
for i in search_results:
    document = i._json
    collection.insert_one(document)
    document['_id'] = str(document['_id'])
    json_users.append(document)
    
print(len(json_users))

100
100


In [27]:

new_users = new_users + json_users
print(len(new_users))

289


In [27]:
# Get all the tweets stored in the collection "tweet_users" in mlab

cursor = collection.find()
print(cursor)

# Obtain json 
json_docs = []
for doc in cursor:
    doc['_id'] = str(doc['_id'])
    json_docs.append(doc)

In [28]:
# Data format (uploaded to mlab in "tweet_users" collection)

print(json.dumps([json_docs[0]], sort_keys=True, indent=4, separators=(',',':')))

[
    {
        "_id":"5bd4a79db4c0e32bc7f902b1",
        "contributors":null,
        "coordinates":null,
        "created_at":"Sat Oct 27 17:59:42 +0000 2018",
        "entities":{
            "hashtags":[],
            "symbols":[],
            "urls":[],
            "user_mentions":[
                {
                    "id":1031950877387620358,
                    "id_str":"1031950877387620358",
                    "indices":[
                        0,
                        11
                    ],
                    "name":"spooky yume\ud83d\udd77",
                    "screen_name":"__yumecchi"
                }
            ]
        },
        "favorite_count":0,
        "favorited":false,
        "geo":null,
        "id":1056244057838641152,
        "id_str":"1056244057838641152",
        "in_reply_to_screen_name":"__yumecchi",
        "in_reply_to_status_id":1056234978965626881,
        "in_reply_to_status_id_str":"1056234978965626881",
        "in_reply_to_user_id":103

In [45]:
# Get all the usernames

user_names = []

for doc in json_users:
    user_names.append(doc['user']['screen_name'])

In [46]:
print(len(user_names))

100


In [30]:
# Number of unique users found

import numpy as np

user_names = set(user_names)
print(len(user_names))

278


In [47]:
# Get 25 tweets of each user until 30th of September of 2018

import time

#MAX_ID = 1046439836939415562 # 30 of September
MAX_ID = 1055786926802898945 # 26 of October

all_tweets = []

for user in user_names:
    
    try:
        new_tweets = api.user_timeline(screen_name=user, count=25, max_id = MAX_ID)
        
    except tweepy.RateLimitError:
        print('Rate limit error with user: %s' % user)
        time.sleep(120)
        continue
    
    except tweepy.TweepError:
        print('Tweep error with user: %s' % user)
        time.sleep(60)
        continue
        
    print(user)
        
    all_tweets.extend(new_tweets)
    
    time.sleep(60)
   

antpogar
DarylMaldonado
menesess_pau
Ura2019
commiesans
Yerbaalawita
Kaytxar
I3Aunizar
GracielaLpez5
Nurya1952
Angelmg90
_asgxv
AlvatrosPubli
thefromthetree
irinaa_ce
AlfredoRstrepo
annasazzle
EfectoCognitivo
morbologo75
igsan_madr
gabinavarros
cmnoya60
gp_victor89
salvaglnet
Mildred12279353
ElChikitoGH
MonWoW_
FerreroEU
ciudadanodmund
angelapeke16
minnervaa
chag_hai
LeiretxoP
firecrackerx
AnaJorgegon
Jusesantos28
RotaDatos
victordejotamar
Postematico
JaraTirado
cristinetavg
Nutriguia
Victor_Corso
CarolinaSarrion
luciagarcia02
aragonradio
juanflm
AjXirivella
inmaCR
AngelLargo1970
Bea_sotelo
ShadowsNessa
Infinito81
DiscosRuidosos
pato_albert
mercedesslopez8
pipio44
vmezadelc
KateAusten5
Egarcamp111
InboxPublicitat
esguilu
SoniaGonzlezSa2
anndrxa_
angusticlavio
elvictorjuan
COMENTANDOGHVI2
Salem_Lombardi
liberal_mirada
anaisleali
LaNaveMadrid
negociosyempren
Themeele
jpa1968fr
jaimitollamas
EJASOabogados
AEHCOS
firelovesrain
JoseLui24493852
cafes_eguia
ElChikitoGH
SenorSherry
JAMonzon
Gi

In [48]:
# Number of tweets from users located in Spain until 30th of September of 2018

print(len(all_tweets))    

2353


In [33]:
# Get tweet id from a known date

n = api.user_timeline(screen_name='nachomartin83', count=30)
for tweet in n:
    print(tweet._json['id'])
    print(tweet._json['created_at'])

1059964276847337472
Wed Nov 07 00:22:31 +0000 2018
1059947383663067137
Tue Nov 06 23:15:23 +0000 2018
1059910388463079424
Tue Nov 06 20:48:23 +0000 2018
1058155246130749442
Fri Nov 02 00:34:04 +0000 2018
1057010726768910338
Mon Oct 29 20:46:10 +0000 2018
1056665912684806144
Sun Oct 28 21:56:00 +0000 2018
1056665156338495488
Sun Oct 28 21:52:59 +0000 2018
1056510174675025920
Sun Oct 28 11:37:09 +0000 2018
1056176058700042240
Sat Oct 27 13:29:29 +0000 2018
1056175305965166592
Sat Oct 27 13:26:30 +0000 2018
1055940103296741378
Fri Oct 26 21:51:53 +0000 2018
1055917265902804992
Fri Oct 26 20:21:08 +0000 2018
1055916951208366081
Fri Oct 26 20:19:53 +0000 2018
1055787168700993536
Fri Oct 26 11:44:11 +0000 2018
1055786926802898945
Fri Oct 26 11:43:13 +0000 2018
1055105876091158528
Wed Oct 24 14:36:58 +0000 2018
1055105362272096257
Wed Oct 24 14:34:55 +0000 2018
1054799457617956865
Tue Oct 23 18:19:22 +0000 2018
1054306255496531969
Mon Oct 22 09:39:34 +0000 2018
1054107552743604225
Sun Oct 21 

In [50]:
# Store all tweets found in mlab "tweets" collection

collection = db['tweets']

#all_tweets = all_tweets[3000:]

for tweet in all_tweets:
    collection.insert_one(tweet._json)